In [7]:
from matplotlib import pyplot as plt

from fit.matrices.create_top_mats import create_px
import numpy as np

from fit.mesh.mesh import Mesh

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
xmesh = np.linspace(0, 10, 10)
ymesh = np.linspace(0, 10, 10)
zmesh = np.linspace(0, 10, 10)
msh = Mesh(xmesh, ymesh, zmesh)

In [11]:
px = create_px(msh.np)
plt.spy(px)

(1999,)
(1999,)
(1999, 1)
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\devin\anaconda3\envs\multiphysics\Lib\site-packages\IPython\core\interactiveshell.py", line 3526, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
    
  File "C:\Users\devin\AppData\Local\Temp\ipykernel_9552\2702872323.py", line 1, in <module>
    px = create_px(msh.np)
    
  File "C:\Users\devin\Repos\Uni\multiphysics\fit\matrices\create_top_mats.py", line 21, in create_px
    return csr_array((vals, (rows, cols)), shape=(n, n))
    
  File "C:\Users\devin\anaconda3\envs\multiphysics\Lib\site-packages\scipy\sparse\_compressed.py", line 52, in __init__
    self._coo_container(arg1, shape=shape, dtype=dtype)
    
  File "C:\Users\devin\anaconda3\envs\multiphysics\Lib\site-packages\scipy\sparse\_coo.py", line 204, in __init__
    self._check()
    
  File "C:\Users\devin\anaconda3\envs\multiphysics\Lib\site-packages\scipy\sparse\_coo.py", line 289, in _check
    if self.nnz > 0:
    
  File "C:\Users\devin\anaconda3\e